# A Trajectory Analysis using the ICESat2 Satellite



**Overview of the steps involved:**

1. Get very high-fidelity orbit trajectory information of the primary satellite:
   - This can be done using a POD-reduced dynamics run in which all empirical accelerations and other parameters are used to compensate for any mismodelled forces.
   - This has already been done for our purposes.

2. Do a run of GEODYN in which the orbit trajectory IS the tracking datatype (PCE).
   - Use this run type to do all density model assessments
   - The residuals will be the difference from the truth.

3. Methods for assessing the density models (using trajectory run type):
  -  General fit (residuals and RMS)
  -  Arc overlap
        - If any arcs overlap, look at how well different density models provide consistency between overlapping arcs
  - Test of prediction
        - Do a fit from t1 to t2 and then predict to some t3.  If the density model is better, the difference between predicted orbit (t2 to t3) and the precise trajectory (PCE data) will contain how well the model is doing.  





## Step 0:  Pre-processing the PCE data into a G2B file

We will construct a `G2B_PCE`file from a set of binary `.rvg` files which contain trajectory output from a reduced-dynamics run of GEODYN. The files will be stitched together to create a the GEODYN-specific, trajectory-based tracking data type (called PCE (Precice-Clock-Error) ) and stored in a G2B file.

Each `.rvg` file contains a **30-hour arc of ICESat2 trajectory data**.  These datasets are the output from a very precise run of GEODYN in which the orbit has been determined very well (*to a few centimeters accuracy*) using a **reduced dynamics technique** (*empirical accelerations and other parameters are adjusted to account for any mismodelled forces)*.

The process that takes place in `pygeodyn` is as follows:
  1. dump the data from each arc into a usable format
  2. chop of the 3 hour padding on the ends to eliminate discontinuities from end effects
  3. stitch together all the files
  4. smooth over any existing discontinuities between arc gaps or maneuvers.
  5. All data is placed in a single `TRAJ.txt` file which is then fed into a Fortran script (`pce_converer.f`) which converts the data into a `G2B` file to be ingested on `fort.40` of the GEODYN run.

### PygeodynPreprocessing


In [1]:
import copy

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_preprocessing/')
from PYGEODYN_Preprocess import PygeodynPreprocessing

path_to_prep_directory = '/data/data_geodyn/inputs/icesat2/pre_processing'
path_to_binaryrvg     = '/data/data_geodyn/inputs/icesat2/pre_processing/traj_files_rvg'

arc1_files = [
              'orbit.1807001.2018.331',
              'orbit.1807001.2018.334',
              'orbit.1807001.2018.335',
              'orbit.1807001.2018.336',
              'orbit.1807001.2018.337',
              'orbit.1807001.2018.338',
              'orbit.1807001.2018.341',
              'orbit.1807001.2018.345',
              'orbit.1807001.2018.348',
              'orbit.1807001.2018.349',
              'orbit.1807001.2018.351',
              'orbit.1807001.2018.352',
              'orbit.1807001.2018.353',
                ]

##### Uncomment the below call to overwrite the G2B data by re-running the pre-processing code
Obj = PygeodynPreprocessing(path_to_binaryrvg, path_to_prep_directory,  arc1_files)
Obj.run_preprocess_PCE()


# break

      Running through the pre-processing procedure...
      STEP 1: Convert RVG binary files to pandas DataFrame...
      Loading and processing 13 files will take approx. 11.81 minutes. 
            Not including unzipping/zipping times

      --- File 1 / 13 
      ----- Unzipping file... orbit.1807001.2018.331
      ----- The file has ~109434 records. Will take ~54 seconds
      ----- End of file
      Zipping file... orbit.1807001.2018.331

      --- File 2 / 13 
      ----- Unzipping file... orbit.1807001.2018.334
      ----- The file has ~109434 records. Will take ~54 seconds
      ----- End of file
      Zipping file... orbit.1807001.2018.334

      --- File 3 / 13 
      ----- Unzipping file... orbit.1807001.2018.335
      ----- The file has ~109434 records. Will take ~54 seconds
      ----- End of file
      Zipping file... orbit.1807001.2018.335

      --- File 4 / 13 
      ----- Unzipping file... orbit.1807001.2018.336
      ----- The file has ~109434 records. Will take ~54

## Step 1: Run GEODYN using Pygeodyn with the ICESat2 configuration

As a reminder, PYGEODYN is called with the `PYGEODYN` class, but modifications to the ICESAT2 configuration are largely controlled through the `Satellite_ICESat2` class in the `PYGEODYN_Satellites.py` file.   Please refer to this file first for file structure setups and modifications to the Pygeodyn structure.
 
### Step 1.1:  Run with MSIS2


In [ ]:
### Identify which arcs you want to run:
arcs_files = [ 
            '2018.313', # 1
            '2018.314', # 2
            '2018.315', # 3
            '2018.316', # 4
            '2018.317', # 5
#             '2018.318', # 6
#             '2018.319', # 7
#             '2018.320', # 8
#             '2018.321', # 9
#             '2018.322', # 10
             ]

#------ A dictionary containing the run parameters ------  
run_params = {} 
run_params['arc']              =   arcs_files
run_params['satellite']        =  'icesat2'  
run_params['den_model']        =  'msis2'  
run_params['SpecialRun_name']  =  '_TrajAnalysis'  
run_params['verbose']          =  False
run_params['action']           =  'run'


In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn


#### ---------------------------------------------
####
#### ----------------- RUN MSIS2.0 --------------- 
####
#### ---------------------------------------------

##### Use copy.deepcopy to copy all levels of dictionary and 
###       allow modification of new variable
run_params1 = copy.deepcopy(run_params)
run_params1['den_model'] =  'msis2'  

# ### Run pyeodyn for the arcs in the above set.
# Obj_Geodyn = Pygeodyn(run_params1)
# Obj_Geodyn.RUN_GEODYN()


### Step  1.2: Run with DTM87 for comparison


In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

#### ---------------------------------------------
####
#### ------------------- RUN DTM ----------------- 
####
#### ---------------------------------------------


##### Use copy.deepcopy to copy all levels of dictionary and 
###       allow modification of new variable
run_params2                      = copy.deepcopy(run_params)
run_params2['den_model']         =  'dtm87'  

### Run pyeodyn for the arcs in the above set.
# Obj_Geodyn = Pygeodyn(run_params2)
# Obj_Geodyn.RUN_GEODYN()


### Step  1.2: Run with Jaachia 71 Model for comparison


In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn


#### ----------------------------------------------
####
#### ----------------- RUN JAACHIA ----------------- 
####
#### ----------------------------------------------

##### Use copy.deepcopy to copy all levels of dictionary and 
###       allow modification of new variable
run_params3 = copy.deepcopy(run_params)
run_params3['den_model']         =  'jaachia71'  


##### Run pyeodyn for the arcs in the above set.
# Obj_Geodyn = Pygeodyn(run_params3)
# Obj_Geodyn.RUN_GEODYN()

    

## Read GEODYN Output using PygeodynRead functionality

### Get MSIS2 Data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

#------ A dictionary containing the run parameters ------  
run_params1 = {} 
run_params1['arc']               =  ['2018.313',
                                     '2018.314',
                                     '2018.315',
                                     '2018.316',
                                     '2018.317',
                                    ] 
run_params1['satellite']         =  'icesat2'  
run_params1['den_model']         =  'msis2'  
run_params1['SpecialRun_name']   =  '_TrajAnalysis'  
run_params1['verbose']           =  False
run_params1['action']            =  'read'
run_params1['request_data']      = ['AdjustedParams',
                 #                    'Trajectory_xyz',
                                    'Trajectory_orbfil',
                                    'Density',
                                    'Residuals_obs',
                                    'Residuals_summary',
                                    'Statistics',
                                   ]

Obj_Geodyn1 = Pygeodyn(run_params1)
Obj_Geodyn1.getData()


### Get DTM87 Data

In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYN import Pygeodyn
import copy


run_params2 = copy.deepcopy(run_params1)

run_params2['den_model']         =  'dtm87'  

Obj_Geodyn2 = Pygeodyn(run_params2)
Obj_Geodyn2.getData()


### Get Jaachia 71 Data

In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYN import Pygeodyn
# import copy


run_params3 = copy.deepcopy(run_params1)
run_params3['den_model']         =  'jaachia71'  

Obj_Geodyn3 = Pygeodyn(run_params3)
Obj_Geodyn3.getData()


## Analyze output

Our analysis has the following output products:

1. Residuals of the POD across many arcs
2. RMS of fit of the POD across many arcs
3. Adjustment of the drag coefficient (drag acceleration to compensate for inaccuracies in the density model.)
4. Check of consistency (in the residuals) across overlapping arc times
5. RMS of the overlapping residual difference (this removes the PCE contribution)
Using the `Orbfil`:
6. From ORBFIL grab the overlapping ephemeris and difference the two models. Compare this against the same of prediction
    - How well does the predicted time period match up with the determined ephemeris (see this in the resids of the two).
7. Calculate and plot the radial component of the trajectory








<!-- Our analysis has the following output products:

**Per Arc:**  

1. Residuals 
2. RMS of fit of the residuals
3. Adjustment of the Drag Coefficient

**Per Arc Overlaps:**  

4. Check of Consistency
5. RMS of the overlap residual difference (this removes the PCE contribution)
6. From ORBFIL grab the overlapping ephemeris and difference the two models. Compare this against the same of prediction

**Trajectory Prediction:**
Use the orbit file to investigate the predicted time period.  

6. How well does the predicted time period match up with the determined ephemeris (residuals).




CONVERT TO RADIAL COMPONENT! -->

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
#                 'staticPlot': True,
                'displaylogo': False,
                'showTips': False,
                })

### 1. Residuals of the POD across many arcs



Resids =  PCE -  POD Trajectory





In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_residual_meas_summary
from PYGEODYNAnalysis_icesat2PCEtrajectory import rms_summary_table


Obj_list = [Obj_Geodyn1,Obj_Geodyn2,Obj_Geodyn3,]
rms_summary_table(Obj_list)



fig = make_subplots(rows=2, cols=1, 
     subplot_titles=(["Mean Residuals per Arc", 'RMS of Fit per Arc']),
     vertical_spacing = 0.1)
fig = plot_residual_meas_summary(fig, Obj_Geodyn2, 0)
fig = plot_residual_meas_summary(fig, Obj_Geodyn3, 1)
fig = plot_residual_meas_summary(fig, Obj_Geodyn1, 2)
fig.show(config=config)



In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_residuals_observed



fig = make_subplots(rows=3, cols=1, 
            subplot_titles=(['X', 'Y', 'Z']),
            vertical_spacing = 0.1,
                       )

fig = plot_residuals_observed(fig, Obj_Geodyn2, 0)
fig = plot_residuals_observed(fig, Obj_Geodyn3, 1)
fig = plot_residuals_observed(fig, Obj_Geodyn1, 2)

fig.update_layout(title="Observation Residuals (PCE - Observed , T.O.R.)")

fig.show(config=config)


In [ ]:
%load_ext autoreload
%autoreload 2
from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_cd_and_percdiff_from_apriori


fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(["Timeseries of Cd", "Percent difference from a priori (Cd=2.2)"]),
    vertical_spacing = 0.08,
    )
fig = plot_cd_and_percdiff_from_apriori(fig,  Obj_Geodyn2, 0)
fig = plot_cd_and_percdiff_from_apriori(fig,  Obj_Geodyn3, 1)
fig = plot_cd_and_percdiff_from_apriori(fig,  Obj_Geodyn1, 2)


fig.show(config=config)

In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_ScaleDensity_with_CdScaleFactor__2

fig = make_subplots(rows=2, cols=1,
                subplot_titles=(["Model Ouptut Density", "Model Density * Cd Scaling Factor"]),
                shared_yaxes=True,
                vertical_spacing = 0.1,
                specs=[
                [{"secondary_y": False}],
                [{"secondary_y": False}], ])


fig = plot_ScaleDensity_with_CdScaleFactor__2(fig,  Obj_Geodyn2, 0, 200)
fig = plot_ScaleDensity_with_CdScaleFactor__2(fig,  Obj_Geodyn3, 1, 200)
fig = plot_ScaleDensity_with_CdScaleFactor__2(fig,  Obj_Geodyn1, 2, 200)

# min_y = 1*1e-16
# max_y = 9*1e-12
# fig.update_yaxes(range=[min_y, max_y], row=1, col=1)
# fig.update_yaxes(range=[min_y, max_y], row=2, col=1)

fig.show(config=config)


### Looking at the arc overlap time:

We want to show the residuals in the overlap time with the PCE data subtracted away.



In [ ]:
####  ARC_OVERLAP_ObsResids_XYZ

%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import ARCOVERLAP_2arcs_ObsResids_XYZ

fig = make_subplots(rows=3, cols=1, 
            subplot_titles=(['X', 'Y', 'Z']),
            vertical_spacing = 0.1,
            specs=[ [{"secondary_y": True }],
                    [{"secondary_y": True }], 
                    [{"secondary_y": True }], ],)

arc1 = '2018.314'  # '2018.314'
arc2 = '2018.315'

fig = ARCOVERLAP_2arcs_ObsResids_XYZ(fig, Obj_Geodyn2, 0, arc1, arc2)
fig = ARCOVERLAP_2arcs_ObsResids_XYZ(fig, Obj_Geodyn3, 1, arc1, arc2)
fig = ARCOVERLAP_2arcs_ObsResids_XYZ(fig, Obj_Geodyn1, 2, arc1, arc2)

fig.show(config=config)


## PCE Data and the Orbit File:

### Residual Component Trajectory:

Convert the Interial XYZ coordinates to the satellite coordinate system (RSW), then plot the radial component.


**Starting Systems:**
- `PCE data`
   - J2000 Coordinate System
   - Inertial satellite State Vector: $[x, y, z, \dot{x}, \dot{y}, \dot{z}]$ (m)
- `ORBFIL data`
   - Mean of year Coordinate System
   - Inertial satellite State Vector: $[x, y, z, \dot{x}, \dot{y}, \dot{z}]$ (m)

**Convert from `XYZ` to `RSW`**



From Vallado pg. 164:
<!-- 

$$ \hat{R} = \frac{\vec{r}}{\lvert\vec{r}\rvert}$$

$$ \hat{W} = \frac{\vec{r} \times \vec{v}}{\lvert \vec{r} \times \vec{v}   \rvert}$$

$$ \hat{S} = \hat{W} \times \hat{R}$$

These allow the transformation:

$$ \vec{r}_{IJK} = \big[\hat{R} \vdots \hat{S} \vdots \hat{W}\big] \,\, \vec{r}_{RSW}  $$

Such that R, S, and W spans the columns of the transformation matrix above.

We then rearrange to get our final equation:

$$ T = \big[\hat{R} \vdots \hat{S} \vdots \hat{W}\big]$$

$$ \vec{r}_{IJK} = T \,\, \vec{r}_{RSW}  $$

$$ T^{-1} \vec{r}_{IJK} = (T^{-1} T)\,\, \vec{r}_{RSW}  $$

$$  \vec{r}_{RSW} =  T^{-1} \vec{r}_{IJK}   $$

 -->

In [ ]:

%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import ARCOVERLAP_2arcs_ObsResids_RSW_radial

fig = make_subplots(rows=2, cols=1, 
            subplot_titles=(['Radial Component', 'Residual (PCE-ORBFIL)']),
            vertical_spacing = 0.2,
            specs=[ [{"secondary_y": False }],
                    [{"secondary_y": False }]],)

arc1 = '2018.314'
arc2 = '2018.315'

fig = ARCOVERLAP_2arcs_ObsResids_RSW_radial(fig, Obj_Geodyn2, 0, arc1, arc2)
fig = ARCOVERLAP_2arcs_ObsResids_RSW_radial(fig, Obj_Geodyn3, 1, arc1, arc2)
fig = ARCOVERLAP_2arcs_ObsResids_RSW_radial(fig, Obj_Geodyn1, 2, arc1, arc2)

fig.show(config=config)


In [ ]:

%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import ARCOVERLAP_2arcs_ObsResids_NTW_intrack

fig = make_subplots(rows=2, cols=1, 
            subplot_titles=(['In-Track Component', 'Residual (PCE-ORBFIL)']),
            vertical_spacing = 0.2,
            specs=[ [{"secondary_y": False }],
                    [{"secondary_y": False }]],)

arc1 = '2018.314'
arc2 = '2018.315'

fig = ARCOVERLAP_2arcs_ObsResids_NTW_intrack(fig, Obj_Geodyn2, 0, arc1, arc2)
fig = ARCOVERLAP_2arcs_ObsResids_NTW_intrack(fig, Obj_Geodyn3, 1, arc1, arc2)
fig = ARCOVERLAP_2arcs_ObsResids_NTW_intrack(fig, Obj_Geodyn1, 2, arc1, arc2)

fig.show(config=config)


<!-- ### Check how well the PCE file and the ORBIT file match up -->